In [922]:
# necessary Modules

import random
from hashlib import sha256, sha3_256
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.backends import default_backend

In [923]:
# define the safe prime p,q and generator g
def generate_dsa_parameters():
    # Generate DSA parameters with 1024-bit p and 160-bit q
    parameters = dsa.generate_parameters(key_size=1024, backend=default_backend())

    # Access the DSA parameters
    p = parameters.parameter_numbers().p
    q = parameters.parameter_numbers().q
    g = parameters.parameter_numbers().g

    print("Generated DSA Parameters:")
    print("p:", p)
    print("q:", q)
    print("g:", g)
    return p,q,g

# test cases
p,q,g = generate_dsa_parameters()

Generated DSA Parameters:
p: 97863186231872680103597815428622196155117545062923008271266272559532673832003745518823626297798797047626653089910653689834993645013310081332092222353203617839426538566376642053223798718631199925385988289219379585815372476962468713693080654592004837654571252818886336230003959709627684524423271992711295834597
q: 742423504357651411106893827268649949659605383023
g: 10815732957907415540589397606728073991567546646163810372209330544565392286405778742415292816223229980887575505382055304249284215622173212117443552525242452470607237856447433061732767956198024433007807094567818607346727475046936122441786533496863162779797548158549179919242287094904388583845799362409858249588


In [924]:
# function to create s (secret key) and v (public key) with given g (generator) and p (prime mod)  
def generate_keys(g, p):
    # secret key s
    s = random.randint(1, p-2)
    # public key = g^s mod p
    v = pow(g, s, p)
    return s, v


# function to generate n random secret and public keys
def generate_n_random_keys(n):
    # a list to store all the pubkey and secrect keys
    keys_list = []
    for _ in range(n):
        # generate random keys
        x_temp, y_temp = generate_keys(g, p)
        # append random keys
        keys_list.append([x_temp, y_temp])
    return keys_list

# function to concat the real key to the random key list and shuffle
def concat_and_shuffle_total_keys(keys_list, x, y):
    # append the real key
    keys_list.append([x, y])
    # do the shuffle
    random.shuffle(keys_list)
    return keys_list

# function to get the real public key's corresponding position
def get_real_key_position(keys_list, y):
    # make the public keys into one list for index
    public_key_list = [key[1] for key in keys_list]
    # return the real public key index
    return public_key_list.index(y)

# function to get the Y, which is the string containing all the public key used in this signature
def get_Y(keys_list):
    # make the public keys into one list for getting the Y
    public_key_list = [key[1] for key in keys_list]
    # generate Y, which is the string containing all the public key used in this signature
    Y = ''.join([str(i) for i in public_key_list])
    return Y

# function to get the initial random value r and R used in the signature
def get_initial_random_r_and_R(g, p):
    r, R = generate_keys(g, p)
    return r, R

# function to generate the signature
def sign(msg, Y, g, r, p, keys_list, x_real, y_real):
    # find the real pubkey's position and +1 to get the next position
    idx = (get_real_key_position(total_keys_list, y_real) + 1) % len(total_keys_list)
    # initial the e_list with None (which represents c in the thesis)
    e_list = [None for _ in range(len(keys_list))]
    # initial the signature list with None 
    sig_list = [None for _ in range(len(keys_list))]
    # calculate the h with Y and using another hash function (thesis 4.1.1)
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16) % p
    h = pow(g, h, p)
    # calculate y wave (thesis 4.1.1)
    y_wave = pow(h, x_real, p)
    # calculate the first e with the random number "r" (thesis 4.1.2)
    e_initial = int(sha256((str(Y) + str(y_wave) +str(msg) + str(pow(g,r,p)) + str(pow(h,r,p))).encode()).hexdigest(), 16) % q
    # place the "e" into e_list
    e_list[idx] = e_initial
    # do the while loop, in order to calculate each position's value until the real public key's position
    while keys_list[idx][1] != y_real:
        # find the "e" used in last term loop
        e_last_one = e_list[idx]
        # generate the random value for the signature (thesis 4.1.3 start)
        _, random_sig = generate_keys(g, q)
        # place it to the position
        sig_list[idx] = random_sig
        # find this term cooresponding y
        temp_y = keys_list[idx][1]
        # do the calculation (thesis 4.1.3 for last two parts)
        # gy part
        calculation = (pow(g, random_sig, p)*pow(temp_y, e_last_one, p)) % p
        # hy part
        calculation_1 = (pow(h, random_sig, p)*pow(y_wave, e_last_one, p)) % p
        # calculate the e for this term (thesis 4.1.3 end)
        temp_e = int(sha256((str(Y) + str(y_wave) +str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16) % q
        # index += 1
        idx = (idx + 1) % len(total_keys_list)
        # place the "e"
        e_list[idx] = temp_e
        
    # q, see paper page 6, part 4, the first sentence. 
    # global q

    # do the real signature (thesis 4.1.4)
    s_real = (r - x_real * e_list[idx]) % (q)
    sig_list[idx] = s_real
    public_key_list = [key[1] for key in keys_list] 
    total_res = {
        'e1': e_list[0],
        'sig_list': sig_list,
        'public_key_list': public_key_list,
        'tag': y_wave,
        'msg': msg,
    }
    return total_res

# define the safe prime p q, and the g (primitive root of p) used in the whole process
p,q,g = generate_dsa_parameters()
msg = 'Hello, Ring Signature'
random_keys_count = 10

# main function
random_keys_list = generate_n_random_keys(random_keys_count)
x_real, y_real = generate_keys(g, p)
total_keys_list = concat_and_shuffle_total_keys(random_keys_list, x_real, y_real)
real_key_index = get_real_key_position(total_keys_list, y_real)
Y = get_Y(total_keys_list)
r, R = get_initial_random_r_and_R(g, q)
total_res = sign(msg, Y, g, r, p, total_keys_list, x_real, y_real)

display(total_res)

Generated DSA Parameters:
p: 126635175215795465367837415027193298430161180221189574714593352911103273513953113366282703761365468967023553739744825437360228357344835783402591792441482559622003916675737543385883804676904785851082901663695472687040763487563401377327316270946942453072023968155936204632624606007515007036743282560046553351803
q: 766282779807081540037178937493993355438656954697
g: 23318449887977951466682752722850851136167549225155587891999317100669874715484747269108778219596119264527679909789784140116164486092239496352924675525107836397415734882592669529543662085352919403075609746518646295541688309651722422357237671475683319555755123710779352553053529720052881954364461312521990893295


{'e1': 521990619451777041683268254959835029223535556440,
 'sig_list': [764176366242671103542543939456307497848077069402,
  663734221616723670011365034090785049061608166806,
  460920348161790369304711502948340632230411367617,
  712279945784175094622739946563333235556307827124,
  639430165985756666624445331745312559475173201811,
  682004602412567842607769424580560874119158279318,
  689596550698153984521945723545643792053525072763,
  602947981338044779603240845806025481604522592150,
  465425678367721031787825984114279551488928305551,
  84354289715650226808067791445472112054557862140,
  411223889666622593178690790637710743608408872608],
 'public_key_list': [37024595828880968182044579909762513028927696125550384333712957180252570304132255467430875764578137045525913810128290140150196065224151334152477219879528328750667746627703548056641303238573494959411673929930544011686308084992007158685887198647932827105606502671725863828064037544225892444317703581133050395743,
  99260476819855008325440909

In [925]:
# This part of code is used for checking the verification process.

def verify(total_res):
    public_key_list = total_res['public_key_list']
    Y = ''.join([str(i) for i in public_key_list])
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16) % p
    h = pow(g, h, p)

    sig_list = total_res['sig_list']
    temp_e_V = e1 = total_res['e1']
    msg = total_res['msg']
    tag = total_res['tag']
    
    for i in range(len(sig_list)):
        temp_sig = sig_list[i]
        temp_y = public_key_list[i]
        calculation = (pow(g, temp_sig, p)*pow(temp_y, temp_e_V, p)) % p
        calculation_1 = (pow(h, temp_sig, p)*pow(tag, temp_e_V, p)) % p
        temp_e_V = int(sha256((str(Y) + str(tag) + str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16) % q

    print('The verification result is:', temp_e_V == e1)
    return temp_e_V == e1



verification_res = verify(total_res)

The verification result is: True


In [926]:
# # This part of code is used for checking the linkability
# # if two signature's tag is identical, then they must come from the same signer

def check_tag(signature_res, signature_res_1):
    return signature_res['tag'] == signature_res_1['tag']

msg_1 = 'new test'
total_res_1 = sign(msg_1, Y, g, r, p, total_keys_list, x_real, y_real)
display(total_res_1)
print("total_res_1['tag'] == total_res['tag']:",check_tag(total_res, total_res_1))

{'e1': 568135892253368490835090704335882297560636561352,
 'sig_list': [6262838785728327819322676428200303022751708120,
  210349405408528875908551521770196974165637418150,
  144785934090231704110720970412924192411387065425,
  554688824719026065251260269726911883189790051364,
  731052403948725780146700882713937424814643901704,
  555297802674034805607293404107749152164787225010,
  403952964385627827120032470049270375381239326764,
  1517400199004906678846024223165399140963122511,
  508935005150989289982203812013283750571390842308,
  679982506666538157402587094282816535029945626585,
  60193115945910285426536891049201017294419015813],
 'public_key_list': [37024595828880968182044579909762513028927696125550384333712957180252570304132255467430875764578137045525913810128290140150196065224151334152477219879528328750667746627703548056641303238573494959411673929930544011686308084992007158685887198647932827105606502671725863828064037544225892444317703581133050395743,
  992604768198550083254409095045

total_res_1['tag'] == total_res['tag']: True
